In [1]:
!yes | pip uninstall torchvision
!pip install torchvision

Found existing installation: torchvision 0.9.1
Uninstalling torchvision-0.9.1:
  Would remove:
    /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision-0.9.1.dist-info/*
    /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision.libs/libcudart.459720b2.so.10.2
    /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision.libs/libz.1328edc3.so.1
    /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision/*
Proceed (y/n)?   Successfully uninstalled torchvision-0.9.1
yes: standard output: Broken pipe
  Using cached torchvision-0.9.1-cp36-cp36m-manylinux1_x86_64.whl (17.4 MB)


# PyTorch Batch Inference
In this notebook, we'll examine how to do batch transform task with PyTorch in Amazon SageMaker. 

First, an image classification model is build on MNIST dataset. Then, we demonstrate batch transform by using SageMaker Python SDK PyTorch framework with different configurations
- `data_type=S3Prefix`: uses all objects that match the specified S3 key name prefix for batch inference.
- `data_type=ManifestFile`: a manifest file containing a list of object keys that you want to batch inference.
- `instance_count>1`: distribute the batch inference dataset to multiple inference instance

For batch transform in TensorFlow in Amazon SageMaker, you can follow other Jupyter notebooks [here](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker_batch_transform)

## Setup
We'll begin with some necessary imports, and get an Amazon SageMaker session to help perform certain tasks, as well as an IAM role with the necessary permissions.

In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from shutil import copyfile
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-batch-inference-script"
print("Bucket:\n{}".format(bucket))

Bucket:
sagemaker-us-west-2-688520471316


## Model Training

Since the main purpose of this notebook is to demonstrate SageMaker PyTorch batch transform, **we reuse this SageMaker Python SDK [PyTorch example](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk/pytorch_mnist) to train a PyTorch model**. It takes around 7 minutes to finish the training.

In [3]:
from torchvision.datasets import MNIST
from torchvision import transforms

local_dir = 'data'
MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]
MNIST(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)


inputs = sagemaker_session.upload_data(path=local_dir, bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist.py",
    role=role,
    framework_version="1.8.0",
    py_version='py3',
    instance_count=3,
    instance_type="ml.c5.2xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"}, # set epochs to a more realistic number for real training
)

estimator.fit({"training": inputs})

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-688520471316/sagemaker/DEMO-pytorch-batch-inference-script
2021-06-04 02:16:50 Starting - Starting the training job...
2021-06-04 02:17:13 Starting - Launching requested ML instancesProfilerReport-1622773010: InProgress
......
2021-06-04 02:18:14 Starting - Preparing the instances for training......
2021-06-04 02:19:14 Downloading - Downloading input data......
2021-06-04 02:20:14 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-06-04 02:20:25,865 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-06-04 02:20:25,867 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-04 02:20:25,875 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropria

# Prepare batch inference data

First, convert the test data into png image; second, upload to your default S3 bucket.

In [4]:
!ls data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [5]:
# untar gz => png

import gzip
import numpy as np
import os

with gzip.open(os.path.join(local_dir, 'MNIST/raw','t10k-images-idx3-ubyte.gz'),  'rb') as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)

In [6]:
len(images)

10000

In [7]:
# randomly sample 100 of test images and upload them to S3

import random
from PIL import Image as im

ids = random.sample(range(len(images)), 100)
ids = np.array(ids, dtype=np.int)
selected_images = images[ids]

image_dir = 'data/images'

if not os.path.exists(image_dir):
    os.makedirs(image_dir)

for i, img in enumerate(selected_images):
    pngimg = im.fromarray(img)
    pngimg.save(os.path.join(image_dir, f'{i}.png'))
    

In [8]:
os.listdir(image_dir)

['78.png',
 '2.png',
 '21.png',
 '59.png',
 '85.png',
 '97.png',
 '37.png',
 '27.png',
 '20.png',
 '47.png',
 '48.png',
 '18.png',
 '68.png',
 '94.png',
 '46.png',
 '62.png',
 '43.png',
 '82.png',
 '33.png',
 '80.png',
 '57.png',
 '83.png',
 '70.png',
 '35.png',
 '10.png',
 '17.png',
 '52.png',
 '26.png',
 '92.png',
 '75.png',
 '56.png',
 '32.png',
 '79.png',
 '15.png',
 '7.png',
 '51.png',
 '9.png',
 '34.png',
 '74.png',
 '5.png',
 '72.png',
 '8.png',
 '93.png',
 '69.png',
 '31.png',
 '96.png',
 '11.png',
 '28.png',
 '24.png',
 '84.png',
 '55.png',
 '81.png',
 '64.png',
 '65.png',
 '39.png',
 '76.png',
 '67.png',
 '66.png',
 '1.png',
 '98.png',
 '77.png',
 '30.png',
 '58.png',
 '71.png',
 '73.png',
 '86.png',
 '89.png',
 '44.png',
 '91.png',
 '99.png',
 '13.png',
 '19.png',
 '14.png',
 '0.png',
 '60.png',
 '49.png',
 '61.png',
 '36.png',
 '38.png',
 '42.png',
 '3.png',
 '88.png',
 '53.png',
 '22.png',
 '63.png',
 '40.png',
 '54.png',
 '87.png',
 '95.png',
 '4.png',
 '25.png',
 '6.png'

In [9]:
inference_prefix = 'batch_transform'
inference_inputs = sagemaker_session.upload_data(path=image_dir, bucket=bucket, key_prefix=inference_prefix)
print("input spec (in this case, just an S3 path): {}".format(inference_inputs))


input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-688520471316/batch_transform


# Create model transformer
Now, we will create a transformer object for handling creating and interacting with Amazon SageMaker transform jobs. We can create the transformer in two ways as shown in the following notebook cells.
- use fitted estimator directly
- first create PyTorchModel from saved model artefect, then create transformer from PyTorchModel object


Here, we implement the `model_fn`, `input_fn`, `predict_fn` and `output_fn` function to override the default [PyTorch inference handler](https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_inference_handler.py). 

It is noted that in `input_fn` function, the inferenced images are encoded as a Python ByteArray. That's why we use `load_from_bytearray` function to load image from `io.BytesIO` then use `PIL.image` to read.

```python
def model_fn(model_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.nn.DataParallel(Net())
    with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
        model.load_state_dict(torch.load(f))
    return model.to(device)

    
def load_from_bytearray(request_body):
    image_as_bytes = io.BytesIO(request_body)
    image = Image.open(image_as_bytes)
    image_tensor = ToTensor()(image).unsqueeze(0)    
    return image_tensor


def input_fn(request_body, request_content_type):
    # if set content_type as 'image/jpg' or 'applicaiton/x-npy', 
    # the input is also a python bytearray
    if request_content_type == 'application/x-image': 
        image_tensor = load_from_bytearray(request_body)
    else:
        print("not support this type yet")
        raise ValueError("not support this type yet")
    return image_tensor


# Perform prediction on the deserialized object, with the loaded model
def predict_fn(input_object, model):
    output = model.forward(input_object)
    pred = output.max(1, keepdim=True)[1]

    return {'predictions':pred.item()}


# Serialize the prediction result into the desired response content type
def output_fn(predictions, response_content_type):
    return json.dumps(predictions)
```

In [10]:
# Use fitted estimator directly
transformer = estimator.transformer(instance_count=1, instance_type="ml.c5.xlarge")

In [11]:
# You can also create a Transformer object from saved model artefect

# get model artefect location by estimator.model_data, or give a S3 key directly
model_artefect_s3_location = estimator.model_data  #'s3://BUCKET/PREFIX/model.tar.gz'

# create PyTorchModel from saved model artefect
pytorch_model = PyTorchModel(
    model_data=model_artefect_s3_location,
    role=role,
    framework_version="1.8.0",
    py_version="py3",
    source_dir=".",
    entry_point="mnist.py",
)

# then create transformer from PyTorchModel object
transformer = pytorch_model.transformer(instance_count=1, instance_type="ml.c5.xlarge")

## Batch inference
Next, we will inference the sampled 100 MNIST images in a batch manner. 

### input images directly from S3 location
We set `S3DataType=S3Prefix` to uses all objects that match the specified S3 key name prefix for batch inference.

In [12]:
transformer.transform(
    data=inference_inputs, data_type="S3Prefix", content_type="application/x-image", wait=True
)

........................2021-06-04 02:26:06,034 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.3.0
TS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 948 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model.mar
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 4
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metrics report format: prometheus
Enable metrics API: true
2021-06-04 02:26:06,064 [INFO ] main org.pytorch.serve.ModelServer - Loading initial mo

### input images by manifest file
First, we generate a manifest file. Then we use the manifest file containing a list of object keys that you want to batch inference. Some key points:
- content_type = 'application/x-image' (!!! here the content_type is for the actual object to be inference, not for the manifest file)
- data_type = 'ManifestFile'
- Manifest file format must follow the format as [this document](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html#SageMaker-Type-S3DataSource-S3DataType) pointed out. We create the manifest file by using jsonlines package.
``` json
[ {"prefix": "s3://customer_bucket/some/prefix/"},
"relative/path/to/custdata-1",
"relative/path/custdata-2",
...
"relative/path/custdata-N"
]
```

In [13]:
!pip install -q jsonlines

In [14]:
import jsonlines
# build image list
manifest_prefix = f"s3://{bucket}/{prefix}/images/"

path = image_dir
img_files = [f for f in listdir(path) if isfile(join(path, f))]

print('img_files\n', img_files)

manifest_content = [{"prefix": manifest_prefix}]
manifest_content.extend(img_files)

print('manifest_content\n', manifest_content)

# write jsonl file
manifest_file = "manifest.json"
with jsonlines.open(manifest_file, mode="w") as writer:
    writer.write(manifest_content)
    
# upload to S3
manifest_obj = sagemaker_session.upload_data(path=manifest_file, key_prefix=prefix)

print('manifest_obj\n', manifest_obj)

img_files
 ['78.png', '2.png', '21.png', '59.png', '85.png', '97.png', '37.png', '27.png', '20.png', '47.png', '48.png', '18.png', '68.png', '94.png', '46.png', '62.png', '43.png', '82.png', '33.png', '80.png', '57.png', '83.png', '70.png', '35.png', '10.png', '17.png', '52.png', '26.png', '92.png', '75.png', '56.png', '32.png', '79.png', '15.png', '7.png', '51.png', '9.png', '34.png', '74.png', '5.png', '72.png', '8.png', '93.png', '69.png', '31.png', '96.png', '11.png', '28.png', '24.png', '84.png', '55.png', '81.png', '64.png', '65.png', '39.png', '76.png', '67.png', '66.png', '1.png', '98.png', '77.png', '30.png', '58.png', '71.png', '73.png', '86.png', '89.png', '44.png', '91.png', '99.png', '13.png', '19.png', '14.png', '0.png', '60.png', '49.png', '61.png', '36.png', '38.png', '42.png', '3.png', '88.png', '53.png', '22.png', '63.png', '40.png', '54.png', '87.png', '95.png', '4.png', '25.png', '6.png', '90.png', '50.png', '12.png', '16.png', '23.png', '29.png', '41.png', '45.png'

In [15]:
# batch transform with manifest file
transform_job = transformer.transform(
    data=manifest_obj, data_type="ManifestFile", content_type="application/x-image", wait=False
)

In [16]:
print('latest transform job \n', transformer.latest_transform_job.name)

latest transform job 
 pytorch-inference-2021-06-04-02-27-00-599


In [17]:
# look at the status of the transform job
import boto3
import pprint as pp

sm_cli = boto3.client('sagemaker')

res = sm_cli.describe_transform_job(
    TransformJobName=transformer.latest_transform_job.name
)

pp.pprint(res)

{'CreationTime': datetime.datetime(2021, 6, 4, 2, 27, 0, 858000, tzinfo=tzlocal()),
 'DataProcessing': {'InputFilter': '$',
                    'JoinSource': 'None',
                    'OutputFilter': '$'},
 'ModelName': 'pytorch-inference-2021-06-04-02-22-15-009',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '870',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 04 Jun 2021 02:27:00 GMT',
                                      'x-amzn-requestid': '66a7c4b7-7c3d-4471-8399-431f35b7afde'},
                      'HTTPStatusCode': 200,
                      'RequestId': '66a7c4b7-7c3d-4471-8399-431f35b7afde',
                      'RetryAttempts': 0},
 'TransformInput': {'CompressionType': 'None',
                    'ContentType': 'application/x-image',
                    'DataSource': {'S3DataSource': {'S3DataType': 'ManifestFile',
                                                    'S3Uri

###  Multiple instance
We use `instance_count > 1` to create multiple inference instances. When a batch transform job starts, Amazon SageMaker initializes compute instances and distributes the inference or preprocessing workload between them. Batch Transform partitions the Amazon S3 objects in the input by key and maps Amazon S3 objects to instances. When you have multiples files, one instance might process input1.csv, and another instance might process the file named input2.csv.

https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html

In [18]:
dist_transformer = estimator.transformer(instance_count=2, instance_type="ml.c4.xlarge")

dist_transformer.transform(
    data=inference_inputs, data_type="S3Prefix", content_type="application/x-image", wait=True
)

## Look at all transform jobs

In [19]:
tjs = sm_cli.list_transform_jobs()['TransformJobSummaries']
for tj in tjs:
    pp.pprint(tj)

{'CreationTime': datetime.datetime(2021, 6, 4, 2, 27, 4, 117000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 6, 4, 2, 27, 4, 117000, tzinfo=tzlocal()),
 'TransformJobArn': 'arn:aws:sagemaker:us-west-2:688520471316:transform-job/pytorch-training-2021-06-04-02-27-01-672',
 'TransformJobName': 'pytorch-training-2021-06-04-02-27-01-672',
 'TransformJobStatus': 'InProgress'}
{'CreationTime': datetime.datetime(2021, 6, 4, 2, 27, 0, 858000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 6, 4, 2, 27, 3, 800000, tzinfo=tzlocal()),
 'TransformJobArn': 'arn:aws:sagemaker:us-west-2:688520471316:transform-job/pytorch-inference-2021-06-04-02-27-00-599',
 'TransformJobName': 'pytorch-inference-2021-06-04-02-27-00-599',
 'TransformJobStatus': 'InProgress'}
{'CreationTime': datetime.datetime(2021, 6, 4, 2, 22, 15, 413000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 6, 4, 2, 26, 24, 700000, tzinfo=tzlocal()),
 'TransformEndTime': datetime.datetime

In [25]:
import pprint as pp

res = sm_cli.describe_transform_job(
    TransformJobName=dist_transformer.latest_transform_job.name
)

pp.pprint(res)

{'CreationTime': datetime.datetime(2021, 6, 4, 2, 27, 4, 117000, tzinfo=tzlocal()),
 'DataProcessing': {'InputFilter': '$',
                    'JoinSource': 'None',
                    'OutputFilter': '$'},
 'Environment': {},
 'ModelName': 'pytorch-training-2021-06-04-02-27-00-982',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '908',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 04 Jun 2021 02:36:18 GMT',
                                      'x-amzn-requestid': 'dfa5913b-85aa-444d-9cad-1882e8eac6cc'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'dfa5913b-85aa-444d-9cad-1882e8eac6cc',
                      'RetryAttempts': 0},
 'TransformEndTime': datetime.datetime(2021, 6, 4, 2, 32, 15, 196000, tzinfo=tzlocal()),
 'TransformInput': {'CompressionType': 'None',
                    'ContentType': 'application/x-image',
                    'DataSource'

In [27]:
import re

def get_bucket_and_prefix(s3_output_path):
    trim = re.sub('s3://', '', s3_output_path)
    bucket, prefix = trim.split('/')
    return bucket, prefix

local_path = 'output' # where to save the output locally

bucket, output_prefix = get_bucket_and_prefix(res['TransformOutput']['S3OutputPath'])
print(bucket, output_prefix)

sagemaker_session.download_data(path=local_path, bucket=bucket, key_prefix=output_prefix)


sagemaker-us-west-2-688520471316 pytorch-training-2021-06-04-02-27-01-672


In [28]:
!ls {local_path}

0.png.out   24.png.out	4.png.out   55.png.out	70.png.out  86.png.out
1.png.out   25.png.out	40.png.out  56.png.out	71.png.out  87.png.out
10.png.out  26.png.out	41.png.out  57.png.out	72.png.out  88.png.out
11.png.out  27.png.out	42.png.out  58.png.out	73.png.out  89.png.out
12.png.out  28.png.out	43.png.out  59.png.out	74.png.out  9.png.out
13.png.out  29.png.out	44.png.out  6.png.out	75.png.out  90.png.out
14.png.out  3.png.out	45.png.out  60.png.out	76.png.out  91.png.out
15.png.out  30.png.out	46.png.out  61.png.out	77.png.out  92.png.out
16.png.out  31.png.out	47.png.out  62.png.out	78.png.out  93.png.out
17.png.out  32.png.out	48.png.out  63.png.out	79.png.out  94.png.out
18.png.out  33.png.out	49.png.out  64.png.out	8.png.out   95.png.out
19.png.out  34.png.out	5.png.out   65.png.out	80.png.out  96.png.out
2.png.out   35.png.out	50.png.out  66.png.out	81.png.out  97.png.out
20.png.out  36.png.out	51.png.out  67.png.out	82.png.out  98.png.out
21.png.out  37.png.out	52.png.out  68

In [23]:
# Inspect the output

import json
for f in os.listdir(local_path):
    path = os.path.join(local_path, f)
    with open(path, 'r') as f:
        pred = json.load(f)
        print(pred)

{'predictions': 9}
{'predictions': 2}
{'predictions': 2}
{'predictions': 0}
{'predictions': 6}
{'predictions': 4}
{'predictions': 8}
{'predictions': 8}
{'predictions': 4}
{'predictions': 9}
{'predictions': 3}
{'predictions': 6}
{'predictions': 3}
{'predictions': 7}
{'predictions': 2}
{'predictions': 1}
{'predictions': 9}
{'predictions': 3}
{'predictions': 5}
{'predictions': 0}
{'predictions': 0}
{'predictions': 7}
{'predictions': 9}
{'predictions': 9}
{'predictions': 0}
{'predictions': 7}
{'predictions': 7}
{'predictions': 5}
{'predictions': 6}
{'predictions': 4}
{'predictions': 6}
{'predictions': 6}
{'predictions': 8}
{'predictions': 0}
{'predictions': 8}
{'predictions': 1}
{'predictions': 2}
{'predictions': 1}
{'predictions': 8}
{'predictions': 3}
{'predictions': 5}
{'predictions': 6}
{'predictions': 2}
{'predictions': 9}
{'predictions': 5}
{'predictions': 4}
{'predictions': 6}
{'predictions': 6}
{'predictions': 4}
{'predictions': 8}
{'predictions': 4}
{'predictions': 9}
{'prediction